<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/triangulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import cv2
import numpy as np

xyz= np.array([[95,10,44],
               [550,0.5,10.5],
               [12,13,43],
               [10,79,76],
               [10,30,50]])

print('original xyz')
print(f'{xyz.round()}')
rvec_1 = np.zeros(3)
tvec_1 = np.zeros(3)
intrinsics = np.array([[664.63297660, 0.00000000, 931.53151205],
                       [0.00000000, 663.83211807, 520.64004697],
                       [0.00000000, 0.00000000, 1.00000000]])

distortion = np.array([-0.40929449, 0.21631278, 0.00001112, 0.00127120, -0.07205712])
image1_points, _ = cv2.projectPoints(xyz, rvec_1, tvec_1, intrinsics, distortion)
print(image1_points)

# apply translation for second image
y_translation = 10
T_1_to_2 = np.array([[1,0,0,0],
                     [0,1,0,y_translation],
                     [0,0,1,0], 
                     [0,0,0,1]])
rvec_2 = np.zeros(3)
tvec_2 = np.zeros(3)
tvec_2[1]=y_translation
image2_points, _ = cv2.projectPoints(xyz, rvec_2, tvec_2, intrinsics, distortion)
print(image2_points)

original xyz
[[ 95.  10.  44.]
 [550.   0.  10.]
 [ 12.  13.  43.]
 [ 10.  79.  76.]
 [ 10.  30.  50.]]
[[[-4.32113632e+03 -3.19899985e+01]]

 [[-5.17605044e+13 -4.69983039e+10]]

 [[ 1.10552020e+03  7.08747172e+02]]

 [[ 9.95108512e+02  1.01498261e+03]]

 [[ 1.04709056e+03  8.65889770e+02]]]
[[[-5.05450146e+03 -7.38887846e+02]]

 [[-5.18170115e+13 -9.88041865e+11]]

 [[ 1.09449093e+03  8.32016552e+02]]

 [[ 9.90087044e+02  1.03073707e+03]]

 [[ 1.03839041e+03  9.45270318e+02]]]


Example-1:<br>
cv2.triangulatePoints is a function of OpenCV that can calculate the coordinates of three-dimensional points. It takes as input two projection matrices and the corresponding coordinates of a point on the image, and outputs the coordinates of the point in 3D space.

Here is a sample code using cv2.triangulatePoints:

In [ ]:
import cv2
import numpy as np

# Create two projection matrices for the left and right cameras
P1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
P2 = np.array([[1, 0, 0, 0.5], [0, 1, 0, 0], [0, 0, 1, 0]])

# Create an array of 2D points in the left and right images
points1 = np.array([[100, 200], [150, 250], [200, 300]])
points2 = np.array([[125, 225], [175, 275], [225, 325]])

# Convert the points to homogeneous coordinates
points1 = np.concatenate((points1, np.ones((points1.shape[0], 1))), axis=1)
points2 = np.concatenate((points2, np.ones((points2.shape[0], 1))), axis=1)

# Triangulate the points to find their 3D coordinates
# points3D = cv2.triangulatePoints(P1, P2, points1.T, points2.T)
points3D = cv2.triangulatePoints(P1[:3], P2[:3], points1.T[:2], points2.T[:2])

# Convert the points back to non-homogeneous coordinates
points3D = points3D / points3D[3]

# Print the 3D coordinates
print(points3D[:3].T)

[[-0.481  0.033  0.000]
 [-0.481  0.030  0.000]
 [-0.480  0.029  0.000]]


Triangulate image points to world points comparing openCV to pure python

In [ ]:
from __future__ import print_function
import numpy as np
import cv2
import time

np.set_printoptions(formatter={'float': '{: 0.3f}'.format})


def triangulate_nviews(P, ip):
    """
    Triangulate a point visible in n camera views.
    P is a list of camera projection matrices.
    ip is a list of homogenised image points. eg [ [x, y, 1], [x, y, 1] ], OR,
    ip is a 2d array - shape nx3 - [ [x, y, 1], [x, y, 1] ]
    len of ip must be the same as len of P
    """
    if not len(ip) == len(P):
        raise ValueError('Number of points and number of cameras not equal.')
    n = len(P)
    M = np.zeros([3*n, 4+n])
    for i, (x, p) in enumerate(zip(ip, P)):
        M[3*i:3*i+3, :4] = p
        M[3*i:3*i+3, 4+i] = -x
    V = np.linalg.svd(M)[-1]
    X = V[-1, :4]
    return X / X[3]


def triangulate_points(P1, P2, x1, x2):
    """
    Two-view triangulation of points in
    x1,x2 (nx3 homog. coordinates).
    Similar to openCV triangulatePoints.
    """
    if not len(x2) == len(x1):
        raise ValueError("Number of points don't match.")
    X = [triangulate_nviews([P1, P2], [x[0], x[1]]) for x in zip(x1, x2)]
    return np.array(X)


# -----------------------------------------------------------------------------
# Data
# -----------------------------------------------------------------------------

# 3 camera projection matrices
P1 = np.array([[5.010e+03, 0.000e+00, 3.600e+02, 0.000e+00],
               [0.000e+00, 5.010e+03, 6.400e+02, 0.000e+00],
               [0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00]])

P2 = np.array([[5.037e+03, -9.611e+01, -1.756e+03, 4.284e+03],
               [2.148e+02,  5.354e+03,  1.918e+02, 8.945e+02],
               [3.925e-01,  7.092e-02,  9.169e-01, 4.930e-01]])

P3 = np.array([[5.217e+03,  2.246e+02,  2.366e+03, -3.799e+03],
               [-5.734e+02,  5.669e+03,  8.233e+02, -2.567e+02],
               [-3.522e-01, -5.839e-02,  9.340e-01,  6.459e-01]])

# 3 corresponding image points - nx2 arrays, n=1
x1 = np.array([[274.128, 624.409]])
x2 = np.array([[239.571, 533.568]])
x3 = np.array([[297.574, 549.260]])

# 3 corresponding homogeneous image points - nx3 arrays, n=1
x1h = np.array([[274.128, 624.409, 1.0]])
x2h = np.array([[239.571, 533.568, 1.0]])
x3h = np.array([[297.574, 549.260, 1.0]])

# 3 corresponding homogeneous image points - nx3 arrays, n=2
x1h2 = np.array([[274.129, 624.409, 1.0], [322.527, 624.869, 1.0]])
x2h2 = np.array([[239.572, 533.568, 1.0], [284.507, 534.572, 1.0]])
x3h2 = np.array([[297.575, 549.260, 1.0], [338.942, 546.567, 1.0]])


# -----------------------------------------------------------------------------
# Test
# -----------------------------------------------------------------------------

print('Triangulate 3d points - units in meters')
# triangulatePoints requires 2xn arrays, so transpose the points
p = cv2.triangulatePoints(P1, P2, x1.T, x2.T)
# however, homgeneous point is returned
p /= p[3]
print('Projected point from openCV:',  p.T)

p = triangulate_nviews([P1, P2], [x1h, x2h])
print('Projected point from 2 camera views:',  p)

p = triangulate_nviews([P1, P2, P3], [x1h, x2h, x3h])
print('Projected point from 3 camera views:',  p)

# cv2 two image points - not homgeneous on input
p = cv2.triangulatePoints(P1, P2, x1h2[:, :2].T, x2h2[:, :2].T)
p /= p[3]
print('Projected points from openCV:\n', p.T)

p = triangulate_points(P1, P2, x1h2, x2h2)
print('Projected point from code:\n',  p)

# -----------------------------------------------------------------------------
# Timing
# -----------------------------------------------------------------------------

t1 = time.time()
for i in range(10000):
    p = cv2.triangulatePoints(P1, P2, x1.T, x2.T)
    p /= p[3]
t2 = time.time()
print('Elapsed time cv2:', t2-t1)

t1 = time.time()
for i in range(10000):
    p = triangulate_nviews([P1, P2], [x1h, x2h])
t2 = time.time()
print('Elapsed time sfm:', t2-t1)

Triangulate 3d points - units in meters
Projected point from openCV: [[-0.035 -0.006  2.022  1.000]]
Projected point from 2 camera views: [-0.034 -0.006  2.023  1.000]
Projected point from 3 camera views: [-0.004  0.062  2.011  1.000]
Projected points from openCV:
 [[-0.035 -0.006  2.022  1.000]
 [-0.015 -0.006  2.018  1.000]]
Projected point from code:
 [[-0.034 -0.006  2.023  1.000]
 [-0.015 -0.006  2.019  1.000]]
Elapsed time cv2: 0.11631059646606445
Elapsed time sfm: 0.6416676044464111


Example 2:<br>
https://www.programcreek.com/python/?code=Huangying-Zhan%2FDF-VO%2FDF-VO-master%2Flibs%2Fgeometry%2Fops_3d.py


In [ ]:
def triangulation(kp1, kp2, T_1w, T_2w):
    """Triangulation to get 3D points
    Args:
        kp1 (Nx2): keypoint in view 1 (normalized)
        kp2 (Nx2): keypoints in view 2 (normalized)
        T_1w (4x4): pose of view 1 w.r.t  i.e. T_1w (from w to 1)
        T_2w (4x4): pose of view 2 w.r.t world, i.e. T_2w (from w to 2)
    Returns:
        X (3xN): 3D coordinates of the keypoints w.r.t world coordinate
        X1 (3xN): 3D coordinates of the keypoints w.r.t view1 coordinate
        X2 (3xN): 3D coordinates of the keypoints w.r.t view2 coordinate
    """
    kp1_3D = np.ones((3, kp1.shape[0]))
    kp2_3D = np.ones((3, kp2.shape[0]))
    kp1_3D[0], kp1_3D[1] = kp1[:, 0].copy(), kp1[:, 1].copy()
    kp2_3D[0], kp2_3D[1] = kp2[:, 0].copy(), kp2[:, 1].copy()
    X = cv2.triangulatePoints(T_1w[:3], T_2w[:3], kp1_3D[:2], kp2_3D[:2])
    X /= X[3]
    X1 = T_1w[:3] @ X
    X2 = T_2w[:3] @ X
    return X[:3], X1, X2 

Example-x <br>
So the setting is given a coordinate system shown below where the z-axis is pointing out of the screen (towards you), the camera focal length is 270 pixels and image resolution is 640x480, then we have an object somewhere in 3D space, and two drones d1 and d2 taking two observations at two different viewpoints, where is d1 is at (6, 3, 2) and the corresponding image coordinate of the object is (320, 280), and that of d2 is (9.5, 4.5, 3) and (160, 408), also the heading of d1 is -20 degrees from the y-axis and that of d2 is +30 degrees from y-axis, the goal is to determine (x, y, z) where the object is at, the drones are hovering over the xy plane
<img src="https://i.stack.imgur.com/vHI9a.png" width="400"/>

Given the information, by letting d1 be the reference frame, we can have the camera intrinsics K = [[270, 0, 320], [0, 270, 240], [0, 0, 1]], the transformation is rotate +50 degrees with z-axis as the rotation axis, and translation t = [3.5, 1.5, 1]


In [ ]:
def pixel2cam(pt, K):
    u = (pt[0] - K[0][2]) / K[0][0]
    v = (pt[1] - K[1][2]) / K[1][1]
    return np.array([u, v], dtype=np.float32)

def triangulate(points_1, points_2, K, R, t):
    T1 = np.array([[1, 0, 0, 0], 
                   [0, 1, 0, 0], 
                   [0, 0, 1, 0]], dtype=np.float32)
    T2 = np.hstack((R, t))
    proj1 = np.matmul(K, T1)
    proj2 = np.matmul(K, T2)
    X = cv2.triangulatePoints(proj1, proj2, points_1, points_2)
    X /= X[3]
    return X

K = np.array([[270, 0, 320], [0, 270, 240], [0, 0, 1]], dtype=np.float32)
# rotate +50 degrees along z axis
rotation_vector = np.array([0, -50 / 180 * np.pi, 0])
R, _ = cv2.Rodrigues(rotation_vector)
t = np.array([[3.5], [1.5], [1]], dtype=np.float)

pt_1 = (320, 280)
pt_2 = (160, 408)

X = triangulate(pt_1, pt_2, K, R, t)

<ipython-input-14-9ceae85dd3e3>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t = np.array([[3.5], [1.5], [1]], dtype=np.float)


Example-x:

In [ ]:
import numpy as np
# Camera projection matrices
P1 = np.eye(4)
P2 = np.array([[ 0.878, -0.01 ,  0.479, -1.995],
            [ 0.01 ,  1.   ,  0.002, -0.226],
            [-0.479,  0.002,  0.878,  0.615],
            [ 0.   ,  0.   ,  0.   ,  1.   ]])
# Homogeneous arrays
a3xN = np.array([[ 0.091,  0.167,  0.231,  0.083,  0.154],
              [ 0.364,  0.333,  0.308,  0.333,  0.308],
              [ 1.   ,  1.   ,  1.   ,  1.   ,  1.   ]])
b3xN = np.array([[ 0.42 ,  0.537,  0.645,  0.431,  0.538],
              [ 0.389,  0.375,  0.362,  0.357,  0.345],
              [ 1.   ,  1.   ,  1.   ,  1.   ,  1.   ]])
# The cv2 method
X = cv2.triangulatePoints( P1[:3], P2[:3], a3xN[:2], b3xN[:2] )
# Remember to divide out the 4th row. Make it homogeneous
X /= X[3]
# Recover the origin arrays from PX
x1 = np.dot(P1[:3],X)
x2 = np.dot(P2[:3],X)
# Again, put in homogeneous form before using them
x1 /= x1[2]
x2 /= x2[2]
 
print ('X\n', X)
print ('x1\n', x1)
print ('x2\n', x2)

X
 [[ 1.003  2.009  3.013  1.004  2.011]
 [ 4.012  4.010  4.017  4.030  4.019]
 [ 11.020  12.029  13.042  12.092  13.055]
 [ 1.000  1.000  1.000  1.000  1.000]]
x1
 [[ 0.091  0.167  0.231  0.083  0.154]
 [ 0.364  0.333  0.308  0.333  0.308]
 [ 1.000  1.000  1.000  1.000  1.000]]
x2
 [[ 0.420  0.537  0.645  0.431  0.538]
 [ 0.389  0.375  0.362  0.357  0.345]
 [ 1.000  1.000  1.000  1.000  1.000]]


Example-3

In [ ]:
import numpy as np
def triangulatePoints( P1, P2, x1, x2 ):
    X = cv2.triangulatePoints( P1[:3], P2[:3], x1[:2], x2[:2] )
    return X/X[3]
# Camera projection matrices
P1 = np.eye(4)
P2 = np.array([[ 0.878, -0.01 ,  0.479, -1.995],
            [ 0.01 ,  1.   ,  0.002, -0.226],
            [-0.479,  0.002,  0.878,  0.615],
            [ 0.   ,  0.   ,  0.   ,  1.   ]])
# Homogeneous arrays
a3xN = np.array([[ 0.091,  0.167,  0.231,  0.083,  0.154],
              [ 0.364,  0.333,  0.308,  0.333,  0.308],
              [ 1.   ,  1.   ,  1.   ,  1.   ,  1.   ]])
b3xN = np.array([[ 0.42 ,  0.537,  0.645,  0.431,  0.538],
              [ 0.389,  0.375,  0.362,  0.357,  0.345],
              [ 1.   ,  1.   ,  1.   ,  1.   ,  1.   ]])

X = triangulatePoints( P1, P2, a3xN, b3xN )
# Remember to divide out the 4th row. Make it homogeneous
X /= X[3]
# Recover the origin arrays from PX
x1 = np.dot(P1[:3],X)
x2 = np.dot(P2[:3],X)
# Again, put in homogeneous form before using them
x1 /= x1[2]
x2 /= x2[2]
 
print ('X\n', X)
print ('x1\n', x1)
print ('x2\n', x2)

X
 [[ 1.003  2.009  3.013  1.004  2.011]
 [ 4.012  4.010  4.017  4.030  4.019]
 [ 11.020  12.029  13.042  12.092  13.055]
 [ 1.000  1.000  1.000  1.000  1.000]]
x1
 [[ 0.091  0.167  0.231  0.083  0.154]
 [ 0.364  0.333  0.308  0.333  0.308]
 [ 1.000  1.000  1.000  1.000  1.000]]
x2
 [[ 0.420  0.537  0.645  0.431  0.538]
 [ 0.389  0.375  0.362  0.357  0.345]
 [ 1.000  1.000  1.000  1.000  1.000]]


Main

In [30]:
!git clone https://github.com/swishswish123/3D_reconstruction.git

Cloning into '3D_reconstruction'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 236 (delta 15), reused 32 (delta 11), pack-reused 198
Receiving objects: 100% (236/236), 118.88 MiB | 27.90 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Updating files: 100% (20/20), done.


In [31]:
%cd 3D_reconstruction

/content/3D_reconstruction


In [43]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':'1n9fJ-a9MQr3BucgcRqIGlF40omruhb4r'}) 
downloaded.GetContentFile('brain.zip')
!unzip -q brain.zip

In [48]:
import os
os.makedirs("assets/random", exist_ok=True)
!cp -rf brain assets/random/

In [35]:
!pip -q install scikit-surgerycore 

  Preparing metadata (setup.py) ... done


In [50]:
!ls assets/random/brain

images	times.npy  vecs.npy


In [56]:
!python reconstruction.py

image 0
image 10
image 20
image 30
image 40
image 50
image 60
image 70
image 80
image 90
image 100
image 110
image 120
/content/3D_reconstruction/reconstruction_utils/utils.py:62: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  euler = rot.as_euler('zyx', degrees=True)
image 130
image 140
image 150
image 160
image 170
image 180
image 190
done


In [57]:
!python plot_reconstruction.py

In [61]:
!python plot_reconstruction_o3d.py

Traceback (most recent call last):
  File "plot_reconstruction_o3d.py", line 19, in <module>
    xyz = np.load(f'{reconstruction_output}/points.npy')
  File "/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py", line 407, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: '/content/3D_reconstruction/reconstructions/online/random/books/points.npy'


In [60]:
!git pull

Already up to date.


In [59]:
!pip -q install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.5/422.5 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.9 MB/s eta 0:00:00
